# Customer Analysis Profit and Sale
In this project, I'll analyse data from data superstore download from kaggle. 

Here are questions I'll try to answer:

1.How long was The last day the customer came to purchase? (In this Analysis I'll use current days from the dataset)

2.How frequncy customer came and purchase?

3.How much customer spend money to purchase?

4.Profit sale of product in each month.

5.Behavior profile of customers.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv("superstore.csv",sep='|',index_col='Row ID')
df.head(2)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2013-152156,2014-11-09T00:00:00Z,2014-11-12T00:00:00Z,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2013-152156,2014-11-09T00:00:00Z,2014-11-12T00:00:00Z,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [3]:
df['Order Date'] = pd.to_datetime(df['Order Date'])


In [4]:
df['Ship Date'] = pd.to_datetime(df['Order Date'])

# Find current day

In [136]:
current_date = df['Order Date'].max()
current_date

Timestamp('2015-12-31 00:00:00+0000', tz='UTC')

In [137]:
(current_date-df['Order Date']).dt.days

Row ID
1        417
2        417
3        566
4        811
5        811
        ... 
9990    1439
9991     307
9992     307
9993     307
9994     240
Name: Order Date, Length: 9994, dtype: int64

In [138]:
def recency(x):
    return (current_date-x.max()).days

In [139]:
recency(df['Order Date'])

0

In [140]:
df.groupby('Customer ID').agg({'Order Date':recency})

,Order Date
Customer ID,
AA-10315,184
AA-10375,19
AA-10480,259
AA-10645,55
AB-10015,415
...,...
XP-21865,43
YC-21895,4
YS-21880,9


# Find Frequency

In [141]:
df.groupby('Customer ID').agg({'Order Date':recency,'Order ID':'nunique'})

,Order Date,Order ID
Customer ID,,
AA-10315,184,5
AA-10375,19,9
AA-10480,259,4
AA-10645,55,6
AB-10015,415,3
...,...,...
XP-21865,43,11
YC-21895,4,5
YS-21880,9,8


In [142]:
df.groupby('Customer ID').agg({'Order Date':recency,'Order ID' : 'nunique','Sales':sum}).rename(columns={'Order Date':'R','Order ID':'F','Sales':'M'})
# R = Recency 
# F = Frequncy
# M = Monetory

,R,F,M
Customer ID,,,
AA-10315,184,5,5563.560
AA-10375,19,9,1056.390
AA-10480,259,4,1790.512
AA-10645,55,6,5086.935
AB-10015,415,3,886.156
...,...,...,...
XP-21865,43,11,2374.658
YC-21895,4,5,5454.350
YS-21880,9,8,6720.444


# Cut data into four part

In [143]:
rfmdata = df.groupby('Customer ID').agg({'Order Date':recency,'Order ID':'nunique','Sales':sum}).rename(columns={'Order Date':'R','Order ID':'F','Sales':'M'})
rfmdata

,R,F,M
Customer ID,,,
AA-10315,184,5,5563.560
AA-10375,19,9,1056.390
AA-10480,259,4,1790.512
AA-10645,55,6,5086.935
AB-10015,415,3,886.156
...,...,...,...
XP-21865,43,11,2374.658
YC-21895,4,5,5454.350
YS-21880,9,8,6720.444


In [144]:
rfmdata.transform(lambda x : pd.qcut(x,4,labels=False))

,R,F,M
Customer ID,,,
AA-10315,3,0,3
AA-10375,0,3,0
AA-10480,3,0,1
AA-10645,1,1,3
AB-10015,3,0,0
...,...,...,...
XP-21865,1,3,2
YC-21895,0,0,3
YS-21880,0,2,3


# Cross tabulation

In [145]:
df.head(1)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2013-152156,2014-11-09 00:00:00+00:00,2014-11-09 00:00:00+00:00,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136


In [146]:
mounlysale = df.set_index('Order Date').groupby(['Category','Sub-Category'])['Sales'].resample('M').sum().reset_index()
mounlysale.head()

,Category,Sub-Category,Order Date,Sales
0,Furniture,Bookcases,2012-01-31 00:00:00+00:00,719.389
1,Furniture,Bookcases,2012-02-29 00:00:00+00:00,290.666
2,Furniture,Bookcases,2012-03-31 00:00:00+00:00,1706.446
3,Furniture,Bookcases,2012-04-30 00:00:00+00:00,308.499
4,Furniture,Bookcases,2012-05-31 00:00:00+00:00,640.631


In [147]:
mounlysale['Order Date'] = mounlysale['Order Date'].dt.to_period('M')

C:\Users\mook\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [148]:
mounlysale.head()

,Category,Sub-Category,Order Date,Sales
0,Furniture,Bookcases,2012-01,719.389
1,Furniture,Bookcases,2012-02,290.666
2,Furniture,Bookcases,2012-03,1706.446
3,Furniture,Bookcases,2012-04,308.499
4,Furniture,Bookcases,2012-05,640.631


In [149]:
mounlysale.pivot_table(index=['Category','Sub-Category'],columns= 'Order Date',values='Sales',aggfunc = sum)

Order Date                     2012-01   2012-02    2012-03   2012-04  \
Category        Sub-Category                                            
Furniture       Bookcases      719.389   290.666   1706.446   308.499   
                Chairs        4187.700   190.458   5713.142  3210.666   
                Furnishings    711.770   392.980    480.128   984.046   
                Tables         333.000  1256.220   6674.240  3441.626   
Office Supplies Appliances     312.580    89.920    502.962   532.580   
                Art            176.990    73.664    413.268   567.916   
                Binders        814.508   339.262   1525.675   985.752   
                Envelopes      194.360     0.000    353.200   408.352   
                Fasteners       37.060    12.350     48.240   101.684   
                Labels          44.746    36.876    173.590    78.992   
                Paper          242.020   138.488   1200.322   625.432   
                Storage       3028.816   142.544   4370.002  3680.012   
                Supplies           NaN   238.620     18.620  4174.354   
Technology      Accessories    827.890  1120.990   1108.934  2545.990   
                Copiers            NaN       NaN        NaN       NaN   
                Machines           NaN       NaN  25314.658   287.968   
                Phones        2315.400   487.520   6087.582  6361.476   

Order Date                     2012-05    2012-06   2012-07    2012-08  \
Category        Sub-Category                                             
Furniture       Bookcases      640.631   759.5736  1487.673   794.2760   
                Chairs        3675.109  6456.0010  6364.940  2590.2420   
                Furnishings    780.166   432.7500   902.622   997.7580   
                Tables        1816.881  5557.8010  2065.816  2938.0705   
Office Supplies Appliances     918.596  2275.2460    81.740  2096.7530   
                Art            288.220   686.1120   256.040   203.6620   
                Binders       4372.304  4275.7930  2934.492  4251.3870   
                Envelopes      214.194    95.5600   136.864   203.3320   
                Fasteners       12.576    45.2920    24.348    85.8700   
                Labels          57.440   607.7260   194.790    41.3760   
                Paper          479.112  1090.1900   796.332  2017.3840   
                Storage        712.902  3849.5140  2452.732  2322.7300   
                Supplies        80.280    27.6040  8243.870   156.9680   
Technology      Accessories    695.490   667.0400  3308.692  1937.7340   
                Copiers       2999.930     0.0000   559.992  1749.9660   
                Machines         0.000   691.7610     0.000     0.0000   
                Phones        5904.456  7077.1640  4135.450  5521.9600   

Order Date                       2012-09   2012-10  ...    2015-03    2015-04  \
Category        Sub-Category                        ...                         
Furniture       Bookcases      2394.4698   616.998  ...   1722.860  2183.8364   
                Chairs        13849.1850  6486.605  ...   5524.914  2099.2860   
                Furnishings    2960.9560   621.812  ...    872.454  2454.9800   
                Tables         4611.8700  4578.832  ...   2477.104  2314.6245   
Office Supplies Appliances     2413.2640   430.792  ...   2439.994   678.7860   
                Art             759.8600   553.526  ...    442.084   592.9860   
                Binders       12743.6160   480.156  ...   2683.200  4942.2930   
                Envelopes       381.3720   340.512  ...    517.016    64.0680   
                Fasteners        95.4780    51.988  ...     24.680    27.7900   
                Labels          381.5360    99.102  ...    224.100   171.2940   
                Paper          1445.3580   990.846  ...   2483.840  1493.2480   
                Storage        8810.2240  3739.586  ...   3815.452  6026.4680   
                Supplies        392.5900   524.620  ...   1687.052   92

# User transactions

In [153]:
mounlysale.head(1)

,Category,Sub-Category,Order Date,Sales
0,Furniture,Bookcases,2012-01,719.389


In [156]:
df[['Order ID','Customer ID','Category','Sub-Category']]

,Order ID,Customer ID,Category,Sub-Category
Row ID,,,,
1,CA-2013-152156,CG-12520,Furniture,Bookcases
2,CA-2013-152156,CG-12520,Furniture,Chairs
3,CA-2013-138688,DV-13045,Office Supplies,Labels
4,US-2012-108966,SO-20335,Furniture,Tables
5,US-2012-108966,SO-20335,Office Supplies,Storage
...,...,...,...,...
9990,CA-2011-110422,TB-21400,Furniture,Furnishings
9991,CA-2014-121258,DB-13060,Furniture,Furnishings
9992,CA-2014-121258,DB-13060,Technology,Phones


In [158]:
df.pivot_table(index='Customer ID',columns=['Category','Sub-Category'],values='Order ID',aggfunc='count',fill_value=0)

Category     Furniture                           Office Supplies              \
Sub-Category Bookcases Chairs Furnishings Tables      Appliances Art Binders   
Customer ID                                                                    
AA-10315             0      0           1      0               2   0       2   
AA-10375             0      0           1      0               0   1       4   
AA-10480             0      0           1      1               0   1       0   
AA-10645             1      3           4      0               0   1       1   
AB-10015             1      1           0      0               0   1       0   
...                ...    ...         ...    ...             ...  ..     ...   
XP-21865             0      0           4      0               2   4       4   
YC-21895             0      0           1      0               0   1       0   
YS-21880             0      1           1      1               0   1       2   
ZC-21910             0      3           3      2               3   3       4   
ZD-21925             0      0           2      0               1   1       1   

Category                                                        Technology  \
Sub-Category Envelopes Fasteners Labels Paper Storage Supplies Accessories   
Customer ID                                                                  
AA-10315             0         1      0     1       1        1           1   
AA-10375             0         0      0     4       2        0           2   
AA-10480             0         0      0     5       2        0           1   
AA-10645             1         0      0     2       3        0           0   
AB-10015             0         0      0     0       2        0           0   
...                ...       ...    ...   ...     ...      ...         ...   
XP-21865             1         1      0     7       2        0           3   
YC-21895             0         0      0     2       2        0           0   
YS-21880             1         0      0     0       4        0           1   
ZC-21910             2         1      0     3       1        0           4   
ZD-21925             0         0      0     1       0        0           0   

Category                              
Sub-Category Copiers Machines Phones  
Customer ID                           
AA-10315           0        0      1  
AA-10375           0        0      1  
AA-10480           0        0      1  
AA-10645           0        0      2  
AB-10015           0        0      1  
...              ...      ...    ...  
XP-21865           0        0      0  
YC-21895           1        0      1  
YS-21880           0        0      0  
ZC-21910           0        0      2  
ZD-21925           0        0      3  

[793 rows x 17 columns]

# Behavior profile

In [162]:
df.head(1)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2013-152156,2014-11-09 00:00:00+00:00,2014-11-09 00:00:00+00:00,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136


In [166]:
df.groupby(['Customer ID','Order ID','Order Date'],as_index=False).agg({'Sales':sum})

,Customer ID,Order ID,Order Date,Sales
0,AA-10315,CA-2011-128055,2012-03-31 00:00:00+00:00,726.548
1,AA-10315,CA-2011-138100,2012-09-15 00:00:00+00:00,29.500
2,AA-10315,CA-2012-121391,2013-10-04 00:00:00+00:00,26.960
3,AA-10315,CA-2013-103982,2014-03-04 00:00:00+00:00,4406.072
4,AA-10315,CA-2014-147039,2015-06-30 00:00:00+00:00,374.480
...,...,...,...,...
5004,ZD-21925,CA-2011-143336,2012-08-27 00:00:00+00:00,244.760
5005,ZD-21925,CA-2013-152471,2014-07-09 00:00:00+00:00,839.944
5006,ZD-21925,CA-2013-167682,2014-04-04 00:00:00+00:00,331.080
5007,ZD-21925,CA-2014-141481,2015-06-12 00:00:00+00:00,61.440


In [190]:
ordermaxsale = df.groupby(['Customer ID','Order ID','Order Date','Sales'],as_index=False)\
    .agg({'Sales':sum})\
    .set_index('Order ID')\
    .groupby(['Customer ID'],as_index=False)\
    .agg({'Sales':'idxmax'})\
    .rename(columns={'Sales':'Order ID'})
ordermaxsale.tail()
# idxmax ------ return index ออกมา

,Customer ID,Order ID
788,XP-21865,CA-2011-114335
789,YC-21895,CA-2011-119375
790,YS-21880,CA-2014-119809
791,ZC-21910,CA-2012-115567
792,ZD-21925,CA-2013-152471


In [191]:
orderecord = df[['Customer ID','Order ID','Order Date']]\
    .drop_duplicates()\
    .sort_values(['Customer ID','Order Date'])
orderecord.head()


,Customer ID,Order ID,Order Date
Row ID,,,
2230,AA-10315,CA-2011-128055,2012-03-31 00:00:00+00:00
7469,AA-10315,CA-2011-138100,2012-09-15 00:00:00+00:00
1300,AA-10315,CA-2012-121391,2013-10-04 00:00:00+00:00
5199,AA-10315,CA-2013-103982,2014-03-04 00:00:00+00:00
1160,AA-10315,CA-2014-147039,2015-06-30 00:00:00+00:00


In [193]:
ordermaxsale['MaxSale'] = 1
ordermaxsale

,Customer ID,Order ID,MaxSale
0,AA-10315,CA-2013-103982,1
1,AA-10375,CA-2013-131065,1
2,AA-10480,CA-2013-114601,1
3,AA-10645,CA-2012-110863,1
4,AB-10015,CA-2013-140935,1
...,...,...,...
788,XP-21865,CA-2011-114335,1
789,YC-21895,CA-2011-119375,1
790,YS-21880,CA-2014-119809,1
791,ZC-21910,CA-2012-115567,1


In [197]:
orderecord.merge(ordermaxsale,on= ['Customer ID','Order ID'],how='left')

,Customer ID,Order ID,Order Date,MaxSale
0,AA-10315,CA-2011-128055,2012-03-31 00:00:00+00:00,NaN
1,AA-10315,CA-2011-138100,2012-09-15 00:00:00+00:00,NaN
2,AA-10315,CA-2012-121391,2013-10-04 00:00:00+00:00,NaN
3,AA-10315,CA-2013-103982,2014-03-04 00:00:00+00:00,1.0
4,AA-10315,CA-2014-147039,2015-06-30 00:00:00+00:00,NaN
...,...,...,...,...
5004,ZD-21925,CA-2011-143336,2012-08-27 00:00:00+00:00,NaN
5005,ZD-21925,CA-2013-167682,2014-04-04 00:00:00+00:00,NaN
5006,ZD-21925,US-2013-147991,2014-05-06 00:00:00+00:00,NaN
5007,ZD-21925,CA-2013-152471,2014-07-09 00:00:00+00:00,1.0


In [205]:
orderecord_1 = orderecord.merge(ordermaxsale,on=['Customer ID','Order ID'],how = 'left')
orderecord_1['M1'] = orderecord_1.groupby('Customer ID')['MaxSale'].shift(-1)
orderecord_1['M2'] = orderecord_1.groupby('Customer ID')['M1'].shift(-1)
orderecord_1['M3'] = orderecord_1.groupby('Customer ID')['M2'].shift(-1)
orderecord_1

,Customer ID,Order ID,Order Date,MaxSale,M1,M2,M3
0,AA-10315,CA-2011-128055,2012-03-31 00:00:00+00:00,NaN,NaN,NaN,1.0
1,AA-10315,CA-2011-138100,2012-09-15 00:00:00+00:00,NaN,NaN,1.0,NaN
2,AA-10315,CA-2012-121391,2013-10-04 00:00:00+00:00,NaN,1.0,NaN,NaN
3,AA-10315,CA-2013-103982,2014-03-04 00:00:00+00:00,1.0,NaN,NaN,NaN
4,AA-10315,CA-2014-147039,2015-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5004,ZD-21925,CA-2011-143336,2012-08-27 00:00:00+00:00,NaN,NaN,NaN,1.0
5005,ZD-21925,CA-2013-167682,2014-04-04 00:00:00+00:00,NaN,NaN,1.0,NaN
5006,ZD-21925,US-2013-147991,2014-05-06 00:00:00+00:00,NaN,1.0,NaN,NaN
5007,ZD-21925,CA-2013-152471,2014-07-09 00:00:00+00:00,1.0,NaN,NaN,NaN


In [206]:
orderecord_1 = orderecord_1.fillna(0)
orderecord_1['Pass record'] = orderecord_1['M1'] + orderecord_1['M2'] + orderecord_1['M3']
orderecord_1

,Customer ID,Order ID,Order Date,MaxSale,M1,M2,M3,Pass record
0,AA-10315,CA-2011-128055,2012-03-31 00:00:00+00:00,0.0,0.0,0.0,1.0,1.0
1,AA-10315,CA-2011-138100,2012-09-15 00:00:00+00:00,0.0,0.0,1.0,0.0,1.0
2,AA-10315,CA-2012-121391,2013-10-04 00:00:00+00:00,0.0,1.0,0.0,0.0,1.0
3,AA-10315,CA-2013-103982,2014-03-04 00:00:00+00:00,1.0,0.0,0.0,0.0,0.0
4,AA-10315,CA-2014-147039,2015-06-30 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
5004,ZD-21925,CA-2011-143336,2012-08-27 00:00:00+00:00,0.0,0.0,0.0,1.0,1.0
5005,ZD-21925,CA-2013-167682,2014-04-04 00:00:00+00:00,0.0,0.0,1.0,0.0,1.0
5006,ZD-21925,US-2013-147991,2014-05-06 00:00:00+00:00,0.0,1.0,0.0,0.0,1.0
5007,ZD-21925,CA-2013-152471,2014-07-09 00:00:00+00:00,1.0,0.0,0.0,0.0,0.0


In [211]:
orderrecord2 = orderecord_1.groupby('Customer ID',as_index=False).agg({'Pass record':sum})
orderrecord2

,Customer ID,Pass record
0,AA-10315,3.0
1,AA-10375,3.0
2,AA-10480,2.0
3,AA-10645,2.0
4,AB-10015,2.0
...,...,...
788,XP-21865,1.0
789,YC-21895,0.0
790,YS-21880,3.0
791,ZC-21910,3.0


In [217]:
selectcus = orderrecord2[orderrecord2['Pass record'] == 3].drop(columns='Pass record')
selectcus

,Customer ID
0,AA-10315
1,AA-10375
5,AB-10060
6,AB-10105
8,AB-10165
...,...
786,VW-21775
787,WB-21850
790,YS-21880
791,ZC-21910


In [224]:
orderecord3 = orderecord_1[orderecord_1['Pass record']==1]\
    .merge(selectcus,on='Customer ID').drop(columns=['MaxSale','Pass record'])
orderecord3

,Customer ID,Order ID,Order Date,M1,M2,M3
0,AA-10315,CA-2011-128055,2012-03-31 00:00:00+00:00,0.0,0.0,1.0
1,AA-10315,CA-2011-138100,2012-09-15 00:00:00+00:00,0.0,1.0,0.0
2,AA-10315,CA-2012-121391,2013-10-04 00:00:00+00:00,1.0,0.0,0.0
3,AA-10375,CA-2012-109939,2013-05-08 00:00:00+00:00,0.0,0.0,1.0
4,AA-10375,CA-2012-114503,2013-11-13 00:00:00+00:00,0.0,1.0,0.0
...,...,...,...,...,...,...
1087,ZC-21910,CA-2012-133452,2013-04-13 00:00:00+00:00,0.0,1.0,0.0
1088,ZC-21910,CA-2012-130365,2013-04-25 00:00:00+00:00,1.0,0.0,0.0
1089,ZD-21925,CA-2011-143336,2012-08-27 00:00:00+00:00,0.0,0.0,1.0
1090,ZD-21925,CA-2013-167682,2014-04-04 00:00:00+00:00,0.0,1.0,0.0


In [233]:
orderpastrec = pd.melt(orderecord3,\
    id_vars=['Customer ID','Order ID','Order Date']\
    ,value_vars=['M1','M2','M3']\
    ,var_name='Month'\
    ,value_name='Value'
    )
orderpastrec

,Customer ID,Order ID,Order Date,Month,value
0,AA-10315,CA-2011-128055,2012-03-31 00:00:00+00:00,M1,0.0
1,AA-10315,CA-2011-138100,2012-09-15 00:00:00+00:00,M1,0.0
2,AA-10315,CA-2012-121391,2013-10-04 00:00:00+00:00,M1,1.0
3,AA-10375,CA-2012-109939,2013-05-08 00:00:00+00:00,M1,0.0
4,AA-10375,CA-2012-114503,2013-11-13 00:00:00+00:00,M1,0.0
...,...,...,...,...,...
3271,ZC-21910,CA-2012-133452,2013-04-13 00:00:00+00:00,M3,0.0
3272,ZC-21910,CA-2012-130365,2013-04-25 00:00:00+00:00,M3,0.0
3273,ZD-21925,CA-2011-143336,2012-08-27 00:00:00+00:00,M3,1.0
3274,ZD-21925,CA-2013-167682,2014-04-04 00:00:00+00:00,M3,0.0


In [239]:
orderpastrec = pd.melt(orderecord3,\
    id_vars=['Customer ID','Order ID','Order Date']\
    ,value_vars=['M1','M2','M3']\
    ,var_name='Month'\
    ,value_name='Value'
    )
orderpastrec = orderpastrec[orderpastrec['Value'] > 0].drop(columns='Value').sort_values(['Customer ID','Order ID'],ascending=False)
orderpastrec.head(10)

,Customer ID,Order ID,Order Date,Month
1091,ZD-21925,US-2013-147991,2014-05-06 00:00:00+00:00,M1
2182,ZD-21925,CA-2013-167682,2014-04-04 00:00:00+00:00,M2
3273,ZD-21925,CA-2011-143336,2012-08-27 00:00:00+00:00,M3
2179,ZC-21910,CA-2012-133452,2013-04-13 00:00:00+00:00,M2
1088,ZC-21910,CA-2012-130365,2013-04-25 00:00:00+00:00,M1
3270,ZC-21910,CA-2011-126802,2012-12-29 00:00:00+00:00,M3
2176,YS-21880,CA-2014-160724,2015-05-06 00:00:00+00:00,M2
3267,YS-21880,CA-2014-135076,2015-04-14 00:00:00+00:00,M3
1085,YS-21880,CA-2014-124828,2015-07-04 00:00:00+00:00,M1
1082,WB-21850,CA-2013-165330,2014-12-12 00:00:00+00:00,M1


In [249]:
saleprofit = df.groupby(['Customer ID','Order ID'],as_index=False).agg({'Sales':sum,'Profit':sum})
saleprofit

,Customer ID,Order ID,Sales,Profit
0,AA-10315,CA-2011-128055,726.548,267.4224
1,AA-10315,CA-2011-138100,29.500,13.2826
2,AA-10315,CA-2012-121391,26.960,7.0096
3,AA-10315,CA-2013-103982,4406.072,-747.1021
4,AA-10315,CA-2014-147039,374.480,96.5050
...,...,...,...,...
5004,ZD-21925,CA-2011-143336,244.760,25.8774
5005,ZD-21925,CA-2013-152471,839.944,56.4925
5006,ZD-21925,CA-2013-167682,331.080,146.8280
5007,ZD-21925,CA-2014-141481,61.440,16.5888


In [250]:
orderpastrec.merge(saleprofit,on=['Customer ID','Order ID'],how = 'inner')

,Customer ID,Order ID,Order Date,Month,Sales,Profit
0,ZD-21925,US-2013-147991,2014-05-06 00:00:00+00:00,M1,16.720,3.3440
1,ZD-21925,CA-2013-167682,2014-04-04 00:00:00+00:00,M2,331.080,146.8280
2,ZD-21925,CA-2011-143336,2012-08-27 00:00:00+00:00,M3,244.760,25.8774
3,ZC-21910,CA-2012-133452,2013-04-13 00:00:00+00:00,M2,910.792,-27.7534
4,ZC-21910,CA-2012-130365,2013-04-25 00:00:00+00:00,M1,349.082,-79.0382
...,...,...,...,...,...,...
1087,AA-10375,CA-2012-114503,2013-11-13 00:00:00+00:00,M2,84.960,6.3720
1088,AA-10375,CA-2012-109939,2013-05-08 00:00:00+00:00,M3,5.248,0.5904
1089,AA-10315,CA-2012-121391,2013-10-04 00:00:00+00:00,M1,26.960,7.0096
1090,AA-10315,CA-2011-138100,2012-09-15 00:00:00+00:00,M2,29.500,13.2826


In [256]:
final = orderpastrec.merge(saleprofit,on=['Customer ID','Order ID'],how = 'inner')\
    .pivot_table(index=['Customer ID'],columns='Month',values=['Sales','Profit'],aggfunc=sum)
final.head()

Profit                        Sales                 
Month              M1        M2        M3       M1      M2       M3
Customer ID                                                        
AA-10315       7.0096   13.2826  267.4224   26.960   29.50  726.548
AA-10375       1.4672    6.3720    0.5904   16.768   84.96    5.248
AB-10060       8.0997  106.5112    8.7138   27.930  334.20   18.540
AB-10105    -204.4458    5.8980 -142.6601  393.165  117.96  671.002
AB-10165      -2.9180   40.9680   -1.0656   46.688  199.64   37.296